# 3D Surface Flattening Tool for Tent Manufacturing
This notebook allows you to flatten 3D surfaces (STL/OBJ) into 2D DXF files for production.

### Instructions:
1. Run the cell below.
2. Upload your 3D file.
3. Select units and flattening method (ARAP recommended for tents).
4. Click 'Flatten & Export' to download the DXF.

In [ ]:
# Install dependencies
!pip install trimesh libigl numpy matplotlib svgwrite ezdxf scipy pyvista ipywidgets

import os
import numpy as np
import trimesh
import igl
import ezdxf
import matplotlib.pyplot as plt
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML

# Logic functions
def get_unit_scale(unit_name):
    return {'mm': 1.0, 'cm': 10.0, 'm': 1000.0, 'inch': 25.4}.get(unit_name.lower(), 1.0)

def find_boundary_loop(start_vertex, adjacency_list):
    loop, curr, prev = [], start_vertex, None
    while True:
        loop.append(curr)
        neighs = adjacency_list[curr]
        nxt = neighs[0] if neighs[0] != prev else neighs[1]
        if nxt == start_vertex: break
        prev, curr = curr, nxt
    return loop

def get_all_bounds(faces):
    bf = igl.boundary_facets(faces)
    adj = {}
    for u, v in bf:
        for x, y in [(u, v), (v, u)]:
            if x not in adj: adj[x] = []
            adj[x].append(y)
    loops, visited = [], set()
    for edge in bf:
        for v in edge:
            if v not in visited:
                l = find_boundary_loop(v, adj)
                loops.append(l); visited.update(l)
    return loops

def flatten(v, f, method='LSCM'):
    b = np.array([0, 1, 2], dtype=np.int64)
    v1, v2, v3 = v[f[0]]
    e1 = v2 - v1; e2 = v3 - v1
    e1u = e1 / np.linalg.norm(e1)
    nu = np.cross(e1, e2) / np.linalg.norm(np.cross(e1, e2))
    vu = np.cross(nu, e1u)
    bc = np.array([[0,0], [np.linalg.norm(e1),0], [np.dot(e2,e1u), np.dot(e2,vu)]])
    if method == 'LSCM':
        res = igl.lscm(v, f, b, bc)
        return res[0] if isinstance(res, tuple) else res
    arap = igl.ARAPData()
    igl.arap_precomputation(v, f, 2, b, arap)
    return igl.arap_solve(bc, arap, igl.lscm(v,f,b,bc)[0])

# UI Setup
upload = widgets.FileUpload(accept='.stl,.obj,.3ds', multiple=False)
unit = widgets.Dropdown(options=['mm', 'cm', 'm', 'inch'], value='mm', description='Units:')
method = widgets.Dropdown(options=['LSCM', 'ARAP'], value='ARAP', description='Method:')
btn = widgets.Button(description="Flatten & Export", button_style='success')
out = widgets.Output()

def process(b):
    with out:
        out.clear_output()
        if not upload.value: print("Upload a file!"); return
        fname = list(upload.value.keys())[0]
        with open(fname, 'wb') as f: f.write(upload.value[fname]['content'])
        mesh = trimesh.load(fname)
        if isinstance(mesh, trimesh.Scene): mesh = mesh.dump(concatenate=True)
        target = sorted(mesh.split(only_watertight=False), key=lambda m: m.area, reverse=True)[0]
        unwrap = flatten(target.vertices, target.faces, method.value)
        bounds = get_all_bounds(target.faces)
        s = get_unit_scale(unit.value)
        dxf = os.path.splitext(fname)[0] + "_flat.dxf"
        doc = ezdxf.new('R2010'); msp = doc.modelspace()
        for bnd in bounds:
            pts = [(p[0]*s, p[1]*s, 0) for p in unwrap[bnd]]
            if pts: pts.append(pts[0]); msp.add_lwpolyline(pts)
        doc.saveas(dxf)
        plt.figure(figsize=(8,8)); [plt.plot(unwrap[bnd][:,0], unwrap[bnd][:,1], 'k-') for bnd in bounds]
        plt.gca().set_aspect('equal'); plt.title("Flat Preview"); plt.show()
        files.download(dxf)

btn.on_click(process)
display(widgets.VBox([upload, unit, method, btn, out]))